# Урок3. Связь бизнес-показателей и DS-метрик

**Ссылки**

1. https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
2. https://en.wikipedia.org/wiki/Receiver_operating_characteristic

**Домашнее задание**

1. обучить несколько разных моделей на наборе данных ССЗ (`train_case2.csv`): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

## Выполнение

#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

#### Загрузка датасета

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [2]:
df = pd.read_csv('materials_lection3/train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


#### Разделим наши данные на тренировочную и тестовую выборки

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', axis=1), 
                                                    df['cardio'], random_state=0)

#### Преобразование

К полям:
- `gender`, `cholesterol` применим OHE-кодирование
- `age`, `height`, `weight`, `ap_hi`, `ap_lo` - standardScaler
- `gluc`, `smoke`, `alco`, `active` - оставим пока как есть

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [
            col for col in pd.get_dummies(X, prefix=self.key).columns
        ]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer = Pipeline([('selector', NumberSelector(key=cont_col)),
                           ('standard', StandardScaler())])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([('selector', ColumnSelector(key=cat_col)),
                                ('ohe', OHEEncoder(key=cat_col))])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([('selector', NumberSelector(key=base_col))])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

#### Обучить несколько разных моделей на наборе данных ССЗ (`train_case2.csv`): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

In [8]:
log_reg = Pipeline([('features', feats),
                    ('classifier', LogisticRegression(random_state=42))])

In [9]:
def scores(classifier):
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score

    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    # расчтаем метрики
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
 
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score]]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [17]:
%%time

results = pd.DataFrame([
    scores(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=8, random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', CatBoostClassifier(max_depth=8, random_state=42))])),

], columns=['CV_score', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier', 'CatBoostClassifier']
results = results.set_index('models')

Learning rate set to 0.054384
0:	learn: 0.6769903	total: 21.7ms	remaining: 21.7s
1:	learn: 0.6624512	total: 50.6ms	remaining: 25.2s
2:	learn: 0.6494634	total: 72.4ms	remaining: 24.1s
3:	learn: 0.6377066	total: 92.1ms	remaining: 22.9s
4:	learn: 0.6274238	total: 111ms	remaining: 22s
5:	learn: 0.6185174	total: 129ms	remaining: 21.4s
6:	learn: 0.6107374	total: 147ms	remaining: 20.8s
7:	learn: 0.6033737	total: 165ms	remaining: 20.4s
8:	learn: 0.5968675	total: 182ms	remaining: 20.1s
9:	learn: 0.5915013	total: 205ms	remaining: 20.3s
10:	learn: 0.5864341	total: 240ms	remaining: 21.6s
11:	learn: 0.5816782	total: 272ms	remaining: 22.4s
12:	learn: 0.5776289	total: 292ms	remaining: 22.2s
13:	learn: 0.5740295	total: 306ms	remaining: 21.5s
14:	learn: 0.5705626	total: 326ms	remaining: 21.4s
15:	learn: 0.5676026	total: 344ms	remaining: 21.2s
16:	learn: 0.5649179	total: 363ms	remaining: 21s
17:	learn: 0.5623110	total: 382ms	remaining: 20.9s
18:	learn: 0.5600778	total: 402ms	remaining: 20.8s
19:	learn: 

162:	learn: 0.5223970	total: 3.49s	remaining: 17.9s
163:	learn: 0.5223175	total: 3.51s	remaining: 17.9s
164:	learn: 0.5222473	total: 3.54s	remaining: 17.9s
165:	learn: 0.5221904	total: 3.55s	remaining: 17.9s
166:	learn: 0.5221467	total: 3.57s	remaining: 17.8s
167:	learn: 0.5220871	total: 3.59s	remaining: 17.8s
168:	learn: 0.5220344	total: 3.61s	remaining: 17.8s
169:	learn: 0.5219625	total: 3.63s	remaining: 17.7s
170:	learn: 0.5217643	total: 3.66s	remaining: 17.7s
171:	learn: 0.5217493	total: 3.67s	remaining: 17.7s
172:	learn: 0.5216982	total: 3.7s	remaining: 17.7s
173:	learn: 0.5215973	total: 3.74s	remaining: 17.7s
174:	learn: 0.5215252	total: 3.76s	remaining: 17.7s
175:	learn: 0.5214514	total: 3.78s	remaining: 17.7s
176:	learn: 0.5213260	total: 3.8s	remaining: 17.7s
177:	learn: 0.5212147	total: 3.82s	remaining: 17.6s
178:	learn: 0.5211247	total: 3.83s	remaining: 17.6s
179:	learn: 0.5210787	total: 3.85s	remaining: 17.6s
180:	learn: 0.5210389	total: 3.87s	remaining: 17.5s
181:	learn: 0.

327:	learn: 0.5092894	total: 7.16s	remaining: 14.7s
328:	learn: 0.5092182	total: 7.18s	remaining: 14.7s
329:	learn: 0.5091041	total: 7.21s	remaining: 14.6s
330:	learn: 0.5090363	total: 7.23s	remaining: 14.6s
331:	learn: 0.5089659	total: 7.25s	remaining: 14.6s
332:	learn: 0.5088712	total: 7.27s	remaining: 14.6s
333:	learn: 0.5087603	total: 7.29s	remaining: 14.5s
334:	learn: 0.5086944	total: 7.32s	remaining: 14.5s
335:	learn: 0.5085242	total: 7.34s	remaining: 14.5s
336:	learn: 0.5085016	total: 7.36s	remaining: 14.5s
337:	learn: 0.5084035	total: 7.39s	remaining: 14.5s
338:	learn: 0.5083484	total: 7.43s	remaining: 14.5s
339:	learn: 0.5082584	total: 7.45s	remaining: 14.5s
340:	learn: 0.5082154	total: 7.47s	remaining: 14.4s
341:	learn: 0.5080902	total: 7.49s	remaining: 14.4s
342:	learn: 0.5079946	total: 7.52s	remaining: 14.4s
343:	learn: 0.5079266	total: 7.54s	remaining: 14.4s
344:	learn: 0.5078766	total: 7.56s	remaining: 14.3s
345:	learn: 0.5078113	total: 7.59s	remaining: 14.3s
346:	learn: 

491:	learn: 0.4970075	total: 10.8s	remaining: 11.2s
492:	learn: 0.4969443	total: 10.8s	remaining: 11.2s
493:	learn: 0.4969091	total: 10.9s	remaining: 11.1s
494:	learn: 0.4968336	total: 10.9s	remaining: 11.1s
495:	learn: 0.4967536	total: 10.9s	remaining: 11.1s
496:	learn: 0.4966933	total: 10.9s	remaining: 11.1s
497:	learn: 0.4966234	total: 10.9s	remaining: 11s
498:	learn: 0.4965629	total: 11s	remaining: 11s
499:	learn: 0.4964441	total: 11s	remaining: 11s
500:	learn: 0.4963687	total: 11s	remaining: 11s
501:	learn: 0.4962966	total: 11s	remaining: 10.9s
502:	learn: 0.4962438	total: 11.1s	remaining: 10.9s
503:	learn: 0.4961774	total: 11.1s	remaining: 10.9s
504:	learn: 0.4960811	total: 11.1s	remaining: 10.9s
505:	learn: 0.4960178	total: 11.1s	remaining: 10.9s
506:	learn: 0.4959394	total: 11.2s	remaining: 10.9s
507:	learn: 0.4958821	total: 11.2s	remaining: 10.8s
508:	learn: 0.4958085	total: 11.2s	remaining: 10.8s
509:	learn: 0.4956811	total: 11.2s	remaining: 10.8s
510:	learn: 0.4955960	total:

654:	learn: 0.4856426	total: 14.5s	remaining: 7.63s
655:	learn: 0.4855435	total: 14.5s	remaining: 7.61s
656:	learn: 0.4855051	total: 14.5s	remaining: 7.59s
657:	learn: 0.4854039	total: 14.6s	remaining: 7.57s
658:	learn: 0.4853450	total: 14.6s	remaining: 7.54s
659:	learn: 0.4852892	total: 14.6s	remaining: 7.52s
660:	learn: 0.4852532	total: 14.6s	remaining: 7.49s
661:	learn: 0.4852150	total: 14.6s	remaining: 7.47s
662:	learn: 0.4851997	total: 14.7s	remaining: 7.45s
663:	learn: 0.4851315	total: 14.7s	remaining: 7.42s
664:	learn: 0.4850825	total: 14.7s	remaining: 7.41s
665:	learn: 0.4850283	total: 14.7s	remaining: 7.38s
666:	learn: 0.4849355	total: 14.7s	remaining: 7.36s
667:	learn: 0.4849103	total: 14.8s	remaining: 7.34s
668:	learn: 0.4848596	total: 14.8s	remaining: 7.32s
669:	learn: 0.4848401	total: 14.8s	remaining: 7.29s
670:	learn: 0.4847436	total: 14.8s	remaining: 7.27s
671:	learn: 0.4846900	total: 14.9s	remaining: 7.25s
672:	learn: 0.4846100	total: 14.9s	remaining: 7.23s
673:	learn: 

...

325:	learn: 0.5091206	total: 7.4s	remaining: 15.3s
326:	learn: 0.5090109	total: 7.43s	remaining: 15.3s
327:	learn: 0.5089455	total: 7.46s	remaining: 15.3s
328:	learn: 0.5088100	total: 7.48s	remaining: 15.3s
329:	learn: 0.5087726	total: 7.5s	remaining: 15.2s
330:	learn: 0.5086842	total: 7.52s	remaining: 15.2s
331:	learn: 0.5085984	total: 7.54s	remaining: 15.2s
332:	learn: 0.5085251	total: 7.56s	remaining: 15.1s
333:	learn: 0.5084835	total: 7.58s	remaining: 15.1s
334:	learn: 0.5084785	total: 7.6s	remaining: 15.1s
335:	learn: 0.5084312	total: 7.63s	remaining: 15.1s
336:	learn: 0.5083613	total: 7.65s	remaining: 15.1s
337:	learn: 0.5082990	total: 7.67s	remaining: 15s
338:	learn: 0.5082195	total: 7.69s	remaining: 15s
339:	learn: 0.5081621	total: 7.71s	remaining: 15s
340:	learn: 0.5080948	total: 7.73s	remaining: 14.9s
341:	learn: 0.5080520	total: 7.75s	remaining: 14.9s
342:	learn: 0.5080462	total: 7.78s	remaining: 14.9s
343:	learn: 0.5080139	total: 7.81s	remaining: 14.9s
344:	learn: 0.5079684

488:	learn: 0.4978867	total: 11.1s	remaining: 11.6s
489:	learn: 0.4977443	total: 11.1s	remaining: 11.6s
490:	learn: 0.4976907	total: 11.1s	remaining: 11.5s
491:	learn: 0.4975550	total: 11.1s	remaining: 11.5s
492:	learn: 0.4974960	total: 11.2s	remaining: 11.5s
493:	learn: 0.4974459	total: 11.2s	remaining: 11.5s
494:	learn: 0.4974431	total: 11.2s	remaining: 11.4s
495:	learn: 0.4973501	total: 11.2s	remaining: 11.4s
496:	learn: 0.4972923	total: 11.2s	remaining: 11.4s
497:	learn: 0.4972278	total: 11.3s	remaining: 11.4s
498:	learn: 0.4971280	total: 11.3s	remaining: 11.3s
499:	learn: 0.4970276	total: 11.3s	remaining: 11.3s
500:	learn: 0.4969348	total: 11.3s	remaining: 11.3s
501:	learn: 0.4968940	total: 11.4s	remaining: 11.3s
502:	learn: 0.4968194	total: 11.4s	remaining: 11.2s
503:	learn: 0.4967714	total: 11.4s	remaining: 11.2s
504:	learn: 0.4967534	total: 11.4s	remaining: 11.2s
505:	learn: 0.4966894	total: 11.5s	remaining: 11.2s
506:	learn: 0.4966488	total: 11.5s	remaining: 11.2s
507:	learn: 

648:	learn: 0.4875292	total: 14.7s	remaining: 7.95s
649:	learn: 0.4875250	total: 14.7s	remaining: 7.92s
650:	learn: 0.4875206	total: 14.7s	remaining: 7.9s
651:	learn: 0.4874835	total: 14.8s	remaining: 7.88s
652:	learn: 0.4874579	total: 14.8s	remaining: 7.86s
653:	learn: 0.4873741	total: 14.8s	remaining: 7.83s
654:	learn: 0.4873286	total: 14.8s	remaining: 7.81s
655:	learn: 0.4872787	total: 14.8s	remaining: 7.78s
656:	learn: 0.4872285	total: 14.9s	remaining: 7.76s
657:	learn: 0.4871453	total: 14.9s	remaining: 7.74s
658:	learn: 0.4870875	total: 14.9s	remaining: 7.72s
659:	learn: 0.4869749	total: 14.9s	remaining: 7.7s
660:	learn: 0.4869131	total: 15s	remaining: 7.67s
661:	learn: 0.4868780	total: 15s	remaining: 7.65s
662:	learn: 0.4867977	total: 15s	remaining: 7.63s
663:	learn: 0.4867520	total: 15s	remaining: 7.6s
664:	learn: 0.4866624	total: 15s	remaining: 7.58s
665:	learn: 0.4866022	total: 15.1s	remaining: 7.56s
666:	learn: 0.4865716	total: 15.1s	remaining: 7.53s
667:	learn: 0.4864653	tot

810:	learn: 0.4785542	total: 18.3s	remaining: 4.27s
811:	learn: 0.4785214	total: 18.4s	remaining: 4.25s
812:	learn: 0.4784810	total: 18.4s	remaining: 4.23s
813:	learn: 0.4783858	total: 18.4s	remaining: 4.21s
814:	learn: 0.4783381	total: 18.4s	remaining: 4.18s
815:	learn: 0.4782392	total: 18.4s	remaining: 4.16s
816:	learn: 0.4781983	total: 18.5s	remaining: 4.14s
817:	learn: 0.4781155	total: 18.5s	remaining: 4.11s
818:	learn: 0.4780862	total: 18.5s	remaining: 4.09s
819:	learn: 0.4780153	total: 18.5s	remaining: 4.07s
820:	learn: 0.4779441	total: 18.6s	remaining: 4.05s
821:	learn: 0.4778838	total: 18.6s	remaining: 4.02s
822:	learn: 0.4778638	total: 18.6s	remaining: 4s
823:	learn: 0.4777956	total: 18.6s	remaining: 3.98s
824:	learn: 0.4777429	total: 18.6s	remaining: 3.96s
825:	learn: 0.4776812	total: 18.7s	remaining: 3.93s
826:	learn: 0.4775667	total: 18.7s	remaining: 3.91s
827:	learn: 0.4775014	total: 18.7s	remaining: 3.89s
828:	learn: 0.4774580	total: 18.7s	remaining: 3.86s
829:	learn: 0.4

976:	learn: 0.4695602	total: 22.2s	remaining: 522ms
977:	learn: 0.4694236	total: 22.2s	remaining: 500ms
978:	learn: 0.4693236	total: 22.2s	remaining: 477ms
979:	learn: 0.4692579	total: 22.3s	remaining: 454ms
980:	learn: 0.4691453	total: 22.3s	remaining: 432ms
981:	learn: 0.4690964	total: 22.3s	remaining: 409ms
982:	learn: 0.4690582	total: 22.3s	remaining: 386ms
983:	learn: 0.4690084	total: 22.3s	remaining: 363ms
984:	learn: 0.4689668	total: 22.4s	remaining: 341ms
985:	learn: 0.4688835	total: 22.4s	remaining: 318ms
986:	learn: 0.4688445	total: 22.4s	remaining: 295ms
987:	learn: 0.4688010	total: 22.4s	remaining: 273ms
988:	learn: 0.4687407	total: 22.5s	remaining: 250ms
989:	learn: 0.4687080	total: 22.5s	remaining: 227ms
990:	learn: 0.4686464	total: 22.5s	remaining: 204ms
991:	learn: 0.4686320	total: 22.5s	remaining: 182ms
992:	learn: 0.4686247	total: 22.5s	remaining: 159ms
993:	learn: 0.4685883	total: 22.6s	remaining: 136ms
994:	learn: 0.4685558	total: 22.6s	remaining: 113ms
995:	learn: 

In [18]:
results

,CV_score,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,,
LogisticRegression,0.787,0.387,0.730,0.647,0.838,0.784
RandomForestClassifier,0.773,0.350,0.719,0.643,0.816,0.771
DecisionTreeClassifier,0.790,0.343,0.734,0.663,0.821,0.792
CatBoostClassifier,0.800,0.363,0.738,0.674,0.815,0.798


#### Вывод

Если судить по ROC_AUC, то наилучшим из представленных моделей является бустинг, но В данной задаче нам важен Recall, чтобы минимизировать случи, когда мы не распознали болезнь. Здесь лучший результат у логистической регрессии. В принципе, имеем дело с очень скудными, достаточно "сырыми" данными и не настроенными моделями. Вполне возможно получить лучший результат.
 

#### (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

При дисбалансе классов precision_recall_curve предпочтительнее. Но при не слишком большом дисбалансе, вполне можно и отмасштабировать roc_auc. В принципе, то же справедливо и для разной цене ошибок 1 и второго рода. Так, в задаче предсказания сердечно-сосудистых заболеваний ложно положительный результат сулит всего лишь обследование у врача, а ложно отрицательный - риск инфаркта, и мы бы хотели больше recall, не посылая, однако, к врачу слишком много людей.
При сильном дисбалансе (100000 объектов, из которых только 100 - класс "1") roc-кривая будет ну очень искаженной. 


True Positive Rate = True Positives / (True Positives + False Negatives)
При малом кол-ве позитивов в данных, False Negatives  будет малым. True Positive Rate, таким образом, будет несколько ближе к 1.

False Positive Rate = False Positives / (False Positives + True Negatives)
Здесь напротив, False Positives будет завышено, поскольку реальных данных 1 класса мало, и для поиска одного тру позитив, модели придется перебрать очень много объектов, имея шанс ошибиться на каждой итеррации. Но поскольку в наших данных большой дисбаланс, много негативов, и True Negatives  так же будет очень велико, а FPR из-за этого очень мало.

Классический пример такого "искажения" - флюорография, где при точности 99%, но при крайне редком распространении туберкулеза, при положительном результате анализа только 1 из 10 человек реально болен, а 9 - ложные срабатывания.

Можно предположить, что данные roc-кривой будут выглядеть "приукрашенными".